In [5]:
# Celda: subir a la raíz del repo automáticamente
import os
from pathlib import Path

def cd_to_project_root(max_up=6):
    p = Path().resolve()
    for _ in range(max_up):
        if (p / "data").exists() and (p / "notebooks").exists():
            os.chdir(p)
            print("📍 Raíz del proyecto:", Path().resolve())
            return
        p = p.parent
    raise RuntimeError("No pude localizar la raíz (carpetas 'data' y 'notebooks').")

cd_to_project_root()
# sanity check
assert Path("data").exists(), "No veo la carpeta data en la ruta actual"


📍 Raíz del proyecto: /Users/test/Desktop/phishing-detector


In [8]:
# Celda 2: preparar carpeta de revisión y listar CSV candidatos
from pathlib import Path

REVIEW_DIR = Path("data/interim/phishing/revision")
REVIEW_DIR.mkdir(parents=True, exist_ok=True)

print("📁 Carpeta de revisión:", REVIEW_DIR.resolve())

# Sugerencia de nombres (puedes usar otros, pero así queda claro)
esperados = [
    "candidatos_es_fuerte.csv",
    "candidatos_es_medio.csv"
]
print("🧭 Sugeridos:", ", ".join(esperados))

# Listar lo que haya ahora mismo
csvs = sorted(REVIEW_DIR.glob("*.csv"))
print(f"🔎 Detectados: {len(csvs)}")
for p in csvs:
    print(" -", p.name)


📁 Carpeta de revisión: /Users/test/Desktop/phishing-detector/data/interim/phishing/revision
🧭 Sugeridos: candidatos_es_fuerte.csv, candidatos_es_medio.csv
🔎 Detectados: 2
 - candidatos_es_fuerte.csv
 - candidatos_es_medio.csv


In [9]:
# Celda 3: Cargar y preparar hoja de revisión manual
import pandas as pd
from pathlib import Path

REVIEW_DIR = Path("data/interim/phishing/revision")
fuentes = {
    "candidatos_es_fuerte.csv": "fuerte",
    "candidatos_es_medio.csv": "medio",
}

dfs = []
for fname, lote in fuentes.items():
    fpath = REVIEW_DIR / fname
    if not fpath.exists():
        print(f"⚠️ No encontrado: {fpath}")
        continue
    df = pd.read_csv(fpath)
    # Normalizamos columnas esperadas
    # (tolerante: si faltan, las crea vacías)
    for col in ["url", "target", "matched_target", "score_es", "score_esplus",
                "fecha_hora_recoleccion", "fuente"]:
        if col not in df.columns:
            df[col] = None
    df["__lote"] = lote
    dfs.append(df[["url","target","matched_target","score_es","score_esplus",
                   "fecha_hora_recoleccion","fuente","__lote"]])

assert len(dfs) > 0, "No se cargó ningún CSV. Revisa nombres y ruta."

rev = pd.concat(dfs, ignore_index=True)

# Deduplicado por URL manteniendo preferencia de 'fuerte' sobre 'medio'
rev["__rank"] = rev["__lote"].map({"fuerte": 0, "medio": 1})
rev = rev.sort_values(["url","__rank"]).drop_duplicates("url", keep="first").drop(columns="__rank")

# Columnas de revisión manual
if "es_espana" not in rev.columns:   # 1 sí / 0 no / -1 dudosa
    rev["es_espana"] = pd.Series(dtype="Int64")
if "categoria" not in rev.columns:   # banca | telco | saas | ecommerce | público | otras
    rev["categoria"] = pd.Series(dtype="string")
if "notas" not in rev.columns:
    rev["notas"] = pd.Series(dtype="string")
if "revisor" not in rev.columns:
    rev["revisor"] = pd.Series(dtype="string")

# Guardado del maestro de revisión (CSV y Parquet para trabajar ágil)
OUT_CSV = REVIEW_DIR / "revision_master.csv"
OUT_PARQ = REVIEW_DIR / "revision_master.parquet"
rev.to_csv(OUT_CSV, index=False)
try:
    rev.to_parquet(OUT_PARQ, index=False)
except Exception as e:
    print("ℹ️ No se pudo guardar Parquet (opcional):", e)

print(f"✅ Maestro creado: {OUT_CSV.name} | filas: {len(rev)}")
print("   columnas:", list(rev.columns))
print("\n👀 Vista previa:")
display(rev.head(10))


✅ Maestro creado: revision_master.csv | filas: 228
   columnas: ['url', 'target', 'matched_target', 'score_es', 'score_esplus', 'fecha_hora_recoleccion', 'fuente', '__lote', 'es_espana', 'categoria', 'notas', 'revisor']

👀 Vista previa:


,url,target,matched_target,score_es,score_esplus,fecha_hora_recoleccion,fuente,__lote,es_espana,categoria,notas,revisor
48,http://buscat.cat/.mix/onlinefreeTonline/onlin...,Other,NaN,0,3,2025-07-28 12:05:48,PhishTank,fuerte,<NA>,<NA>,<NA>,<NA>
47,http://caixacapitalrisc.send2sign.es/login,Caixa,NaN,1,3,2025-07-28 12:05:48,PhishTank,fuerte,<NA>,<NA>,<NA>,<NA>
120,http://cn83141.tw1.ru/DGT,Other,NaN,0,2,2025-07-28 12:05:48,PhishTank,medio,<NA>,<NA>,<NA>,<NA>
53,http://f1150151.xsph.ru/Orange,Other,orange,1,2,2025-07-28 12:05:48,PhishTank,medio,<NA>,<NA>,<NA>,<NA>
52,http://f1150151.xsph.ru/Orange/login.php,Other,orange,1,2,2025-07-28 12:05:48,PhishTank,medio,<NA>,<NA>,<NA>,<NA>
191,http://orangeeurope89.wixsite.com/verification,Other,orange,1,2,2025-07-28 12:05:48,PhishTank,medio,<NA>,<NA>,<NA>,<NA>
41,http://serviziosicuro.es/lo/apps/private.php?o...,Other,NaN,1,3,2025-07-28 12:05:48,PhishTank,fuerte,<NA>,<NA>,<NA>,<NA>
40,http://serviziosicuro.es/lo/apps/private.php?o...,Other,NaN,1,3,2025-07-28 12:05:48,PhishTank,fuerte,<NA>,<NA>,<NA>,<NA>
42,http://serviziosicuro.es/lo/apps/private.php?o...,Other,NaN,1,3,2025-07-28 12:05:48,PhishTank,fuerte,<NA>,<NA>,<NA>,<NA>
28,https://0x.lfyabbhdluzu.es/tmfu/,Other,NaN,1,3,2025-07-28 12:05:48,PhishTank,fuerte,<NA>,<NA>,<NA>,<NA>
